In [26]:
# to open the right environment run:
# conda activate DEBIAS

In [27]:
from debiasm import DebiasMClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

In [127]:
np.random.seed(123)
n_samples = 96*5
n_batches = 5
n_features = 100

## the read count matrix
X = ( np.random.rand(n_samples, n_features) *1000).astype(int)

## the labels
y = np.random.rand(n_samples)>0.5

## the batches
batches = ( np.random.rand(n_samples) * n_batches ).astype(int)

In [129]:
type(y)

numpy.ndarray

In [115]:
batches

array([4, 4, 3, 3, 1, 2, 0, 1, 1, 4, 0, 2, 1, 4, 1, 1, 0, 4, 4, 4, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 3, 3, 0, 0, 2, 4, 2, 1, 4, 4, 4, 1, 1, 4, 2,
       1, 3, 4, 2, 4, 3, 4, 4, 2, 3, 1, 2, 2, 1, 1, 4, 3, 0, 3, 3, 0, 1,
       4, 0, 3, 3, 3, 2, 4, 1, 1, 0, 0, 3, 0, 4, 3, 1, 3, 2, 1, 3, 2, 4,
       3, 0, 2, 2, 1, 4, 4, 0, 4, 2, 1, 4, 1, 2, 2, 0, 2, 3, 3, 3, 1, 2,
       1, 2, 1, 2, 1, 3, 4, 4, 4, 0, 4, 4, 1, 4, 2, 2, 0, 3, 1, 0, 0, 3,
       3, 2, 4, 4, 3, 4, 4, 1, 3, 4, 2, 4, 4, 0, 4, 4, 2, 2, 2, 0, 2, 1,
       3, 4, 1, 4, 4, 1, 4, 1, 0, 2, 2, 3, 0, 4, 4, 4, 4, 4, 0, 0, 1, 0,
       4, 0, 1, 2, 1, 2, 2, 1, 3, 4, 3, 3, 4, 0, 3, 4, 0, 1, 3, 3, 0, 3,
       4, 4, 0, 3, 1, 3, 4, 3, 2, 2, 4, 0, 0, 4, 4, 4, 2, 4, 1, 3, 4, 0,
       3, 0, 0, 4, 3, 2, 2, 4, 4, 0, 2, 3, 1, 4, 0, 0, 1, 1, 1, 0, 4, 2,
       2, 4, 4, 1, 4, 2, 0, 4, 2, 2, 1, 3, 1, 3, 2, 4, 1, 2, 1, 0, 4, 1,
       3, 0, 2, 0, 2, 2, 2, 1, 1, 4, 1, 0, 4, 1, 0, 1, 3, 2, 0, 4, 0, 0,
       3, 0, 4, 1, 1, 3, 0, 3, 2, 3, 4, 4, 1, 3, 2,

In [110]:
## we assume the batches are numbered ints starting at '0',
## and they are in the first column of the input X matrices

X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch[:10, :10]

array([[4, 6, 2, 2, 5, 7, 4, 9, 6, 4],
       [4, 5, 6, 1, 1, 3, 6, 8, 5, 8],
       [3, 5, 0, 6, 9, 7, 5, 1, 6, 6],
       [3, 0, 7, 0, 0, 2, 8, 3, 5, 5],
       [1, 4, 2, 9, 3, 9, 0, 7, 4, 4],
       [2, 2, 3, 8, 1, 8, 4, 4, 3, 9],
       [0, 7, 7, 9, 8, 4, 6, 4, 2, 3],
       [1, 9, 2, 7, 0, 8, 0, 7, 9, 5],
       [1, 3, 5, 0, 8, 9, 5, 2, 9, 3],
       [4, 8, 2, 2, 1, 9, 1, 7, 9, 6]])

In [111]:
## set the valdiation batch to '4'
val_inds = batches==4
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
y_train, y_val = y[~val_inds], y[val_inds]

In [112]:
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training

dmc.fit(X_train, y_train)
print('finished training!')

c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass 

finished training!


In [102]:
X_debiassed = dmc.transform(X_with_batch)
X_debiassed[:5, :5]

array([[0.01455441, 0.0053293 , 0.00494055, 0.01203451, 0.01376452],
       [0.01066997, 0.01234352, 0.00228306, 0.0028241 , 0.0061122 ],
       [0.01086837, 0.00140471, 0.0145228 , 0.02113101, 0.01631679],
       [0.00031816, 0.01521746, 0.00015438, 0.00176728, 0.0047343 ],
       [0.00968819, 0.00628536, 0.01724374, 0.00602919, 0.01836893]],
      dtype=float32)

## My own Data ##

In [ ]:
np.random.seed(123)
log_DEBIAS = pd.read_csv("./csv_files/log_DEBIAS-M.csv")
data = log_DEBIAS.iloc[:,3:len(log_DEBIAS)]

In [133]:
## the read count matrix and "batches" a.k.a sample_IDs
X = data.iloc[:,2:len(data)]
X = X.to_numpy()

## the labels
y =  data["case"].to_numpy() # the 0s and 1s... make true (case/1) false(control/0)

# batches
# batches = data.iloc[:,1:2]
# batches = batches.to_numpy()

# batches
batches = data.iloc[:, 1]
batches  = batches.to_numpy().astype(int)
batches

array([ 0,  0,  0, ..., 14, 14, 14])

In [134]:
## we assume the batches are numbered ints starting at '0',
## and they are in the first column of the input X matrices

# run this one without editing
# X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch[:10, :10]
X_with_batch


array([ True,  True,  True, ..., False, False, False])

In [135]:
## set the valdiation batch to '4'

# run this one without editing
val_inds = batches==4
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
y_train, y_val = y[~val_inds], y[val_inds]

In [123]:
print((X_train))

array([False, False, False, ..., False, False, False])

In [136]:
# run this one without editing
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training

dmc.fit(X_train, y_train)
print('finished training!')

c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass 

finished training!


In [137]:
roc_auc_score(y_val, dmc.predict_proba(X_val)[:, 1]) 

0.6901857759429537

In [140]:
# run this one without editing

X_debiassed = dmc.transform(X_with_batch)
X_debiassed[:5, :5]

array([0., 0., 0., ..., 0., 0., 0.])

In [165]:
cols = list(data.columns[2:len(data)])

debiased_df = pd.DataFrame(data = X_debiassed,
                           columns = cols)

first_cols = log_DEBIAS.iloc[:, 0:3]

frames = [first_cols, debiased_df]

merged = pd.concat(frames, axis=1)

merged.to_csv("./csv_files/debiased_lognorm.csv", index = False)


In [161]:
debiased_df

,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,X.Eubacterium..eligens.group,X.Eubacterium..fissicatena.group,X.Eubacterium..hallii.group,X.Eubacterium..nodatum.group,X.Eubacterium..oxidoreducens.group,...,Wohlfahrtiimonas,Xanthobacter,Xanthomonas,XBB1006,Xylophilus,Yersinia,Yokenella,Z20,Zavarzinella,Zymomonas
0,0.0,0.0,0.000000,0.000000,0.000000,0.016978,0.00000,0.020180,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.000000,0.012136,0.00000,0.012248,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.022076,0.00000,0.009442,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.009049,0.000000,0.000000,0.015326,0.00000,0.014339,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.012071,0.00000,0.007961,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3597,0.0,0.0,0.000000,0.026908,0.000000,0.009101,0.00483,0.017660,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3598,0.0,0.0,0.000000,0.000000,0.002795,0.005926,0.00000,0.008930,0.00417,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3599,0.0,0.0,0.000000,0.000000,0.000000,0.007910,0.00000,0.004052,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3600,0.0,0.0,0.000000,0.000000,0.000000,0.033113,0.00000,0.005191,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
log_DEBIAS

,sample_name,Phenotype,Study_ID,case,ID,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,...,Wohlfahrtiimonas,Xanthobacter,Xanthomonas,XBB1006,Xylophilus,Yersinia,Yokenella,Z20,Zavarzinella,Zymomonas
0,10959.GBR00561.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
1,10959.GBR00567.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
2,10959.GBR00568.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3,10959.GBR00571.2011,1,10959,True,0,0.0,0.0,1.614424,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
4,10959.GBR00623.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3597,15006.9C1,1,15006,True,14,0.0,0.0,0.000000,0.529216,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3598,15006.9F1,0,15006,False,14,0.0,0.0,0.000000,0.000000,0.717811,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3599,15006.9M1,0,15006,False,14,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3600,15006.9SK1,0,15006,False,14,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
